[CDC Firearm Mortality](https://www.cdc.gov/nchs/state-stats/deaths/firearms.html?CDC_AAref_Val=https://www.cdc.gov/nchs/pressroom/sosmap/firearm_mortality/firearm.htm)

In [48]:
import requests
import pandas as pd

BASE = "https://data.cdc.gov/resource/fpsi-y8tj.json"

def fetch_firearm_state(period="2023", intent="FA_Deaths"):
    params = {
        "$select": "name as state, geoid, intent, period, rate, count_sup, data_as_of",
        "$where": f"period='{period}' AND intent='{intent}'",
        "$order": "state",
        "$limit": 50000
    }
    r = requests.get(BASE, params=params, timeout=60)
    r.raise_for_status()
    df = pd.DataFrame(r.json())
    if not df.empty and "rate" in df.columns:
        df["rate"] = pd.to_numeric(df["rate"], errors="coerce")
    return df

# All firearm deaths by state for 2023
df_all = fetch_firearm_state(period="2023", intent="FA_Deaths")
print("All firearm deaths, 2023:")
print(df_all.head(), "\nRows:", len(df_all))

# Obtain Homicide and Suicde Rates
df_homicide = fetch_firearm_state(period="2023", intent="FA_Homicide")
df_suicide = fetch_firearm_state(period="2023", intent="FA_Suicide")

# Combinne each query
combined = (
    df_all.merge(df_homicide[["state","rate"]].rename(columns={"rate":"fa_homicide_rate"}), on="state", how="left")
          .merge(df_suicide[["state","rate"]].rename(columns={"rate":"fa_suicide_rate"}), on="state", how="left")
)
print("\nCombined firearm mortality by state (all, homicide, suicide):")
print(combined.head())

combined.to_csv("firearm_mortality_by_state_2023.csv", index=False)


All firearm deaths, 2023:
        state geoid     intent period  rate count_sup               data_as_of
0     Alabama    01  FA_Deaths   2023  25.3      1292  2025-09-05T00:00:00.000
1      Alaska    02  FA_Deaths   2023  24.0       176  2025-09-05T00:00:00.000
2     Arizona    04  FA_Deaths   2023  19.1      1419  2025-09-05T00:00:00.000
3    Arkansas    05  FA_Deaths   2023  21.9       671  2025-09-05T00:00:00.000
4  California    06  FA_Deaths   2023   8.2      3209  2025-09-05T00:00:00.000 
Rows: 51

Combined firearm mortality by state (all, homicide, suicide):
        state geoid     intent period  rate count_sup  \
0     Alabama    01  FA_Deaths   2023  25.3      1292   
1      Alaska    02  FA_Deaths   2023  24.0       176   
2     Arizona    04  FA_Deaths   2023  19.1      1419   
3    Arkansas    05  FA_Deaths   2023  21.9       671   
4  California    06  FA_Deaths   2023   8.2      3209   

                data_as_of  fa_homicide_rate  fa_suicide_rate  
0  2025-09-05T00:00:

[World Population Review](https://worldpopulationreview.com/state-rankings/strictest-gun-laws-by-state)

In [49]:
df_laws = pd.read_csv("strictest-gun-laws-by-state-2025.csv")

In [50]:
# If you have different spellings, normalize IN PLACE without creating a key column
combined["state"] = combined["state"].str.strip().replace({"Washington, D.C.": "District of Columbia"})
df_laws["state"] = df_laws["state"].str.strip().replace({"Washington, D.C.": "District of Columbia"})

# Keep only the grade column from df_laws, then merge on 'state'
laws_keep = df_laws[["state", "StrictestGunLawsGiffordGrade_2024"]].drop_duplicates("state")

merged = combined.merge(laws_keep, on="state", how="left")

# Optional: see which states did not match
missing = sorted(set(merged["state"]) - set(laws_keep["state"]))
if missing:
    print("No grade found for:", missing)

merged.head()

No grade found for: ['District of Columbia']


,state,geoid,intent,period,rate,count_sup,data_as_of,fa_homicide_rate,fa_suicide_rate,StrictestGunLawsGiffordGrade_2024
0,Alabama,01,FA_Deaths,2023,25.3,1292,2025-09-05T00:00:00.000,12.4,12.1,F
1,Alaska,02,FA_Deaths,2023,24.0,176,2025-09-05T00:00:00.000,6.4,16.4,F
2,Arizona,04,FA_Deaths,2023,19.1,1419,2025-09-05T00:00:00.000,5.5,12.8,F
3,Arkansas,05,FA_Deaths,2023,21.9,671,2025-09-05T00:00:00.000,8.6,12.8,F
4,California,06,FA_Deaths,2023,8.2,3209,2025-09-05T00:00:00.000,3.7,4.2,A


In [51]:
# normalize and extract a clean letter grade like A, A-, A+, B, B+, B-, ..., F
g = (
    merged["StrictestGunLawsGiffordGrade_2024"]
      .astype(str).str.upper().str.strip()
      .str.extract(r'(A\+|A\-|A|B\+|B\-|B|C\+|C\-|C|D\+|D\-|D|F)')[0]
)

grade_to_likert = {
    "A+": "very strong", "A": "very strong", "A-": "very strong",
    "B+": "strong", "B": "strong", "B-": "strong",
    "C+": "neutral", "C": "neutral", "C-": "neutral",
    "D+": "weak", "D": "weak", "D-": "weak",
    "F": "very weak",
}

merged["StrictestGunLawsGiffordGrade_2024"] = g.map(grade_to_likert)

likert_order = ["very weak", "weak", "neutral", "strong", "very strong"]
cat = pd.CategoricalDtype(categories=likert_order, ordered=True)
merged["StrictestGunLawsGiffordGrade_2024"] = merged["StrictestGunLawsGiffordGrade_2024"].astype(cat)

unmapped = merged[merged["StrictestGunLawsGiffordGrade_2024"].isna()]
if not unmapped.empty:
    print("Unmapped grades found in StrictestGunLawsGiffordGrade_2024:")
    print(unmapped[["state"]].assign(original=merged["StrictestGunLawsGiffordGrade_2024"]))

In [52]:
merged = merged.rename(columns={'StrictestGunLawsGiffordGrade_2024':'gun_laws'})

In [53]:
merged

,state,geoid,intent,period,rate,count_sup,data_as_of,fa_homicide_rate,fa_suicide_rate,gun_laws
0,Alabama,01,FA_Deaths,2023,25.3,1292,2025-09-05T00:00:00.000,12.4,12.1,very weak
1,Alaska,02,FA_Deaths,2023,24.0,176,2025-09-05T00:00:00.000,6.4,16.4,very weak
2,Arizona,04,FA_Deaths,2023,19.1,1419,2025-09-05T00:00:00.000,5.5,12.8,very weak
3,Arkansas,05,FA_Deaths,2023,21.9,671,2025-09-05T00:00:00.000,8.6,12.8,very weak
4,California,06,FA_Deaths,2023,8.2,3209,2025-09-05T00:00:00.000,3.7,4.2,very strong
5,Colorado,08,FA_Deaths,2023,17.3,1019,2025-09-05T00:00:00.000,4.4,12.3,very strong
6,Connecticut,09,FA_Deaths,2023,6.2,225,2025-09-05T00:00:00.000,2.9,3.1,very strong
7,Delaware,10,FA_Deaths,2023,12.0,124,2025-09-05T00:00:00.000,4.2,7.4,very strong
8,District of Columbia,11,FA_Deaths,2023,33.1,225,2025-09-05T00:00:00.000,31.1,1.5,very strong
9,Florida,12,FA_Deaths,2023,14.4,3253,2025-09-05T00:00:00.000,4.7,9.3,neutral


In [54]:
merged.to_csv("gun_data.csv")